# Simple Substitution
`w266 Final Project: Crosslingual Word Embeddings`

The code in this notebook was used to develop an algorithm to generate crosslingual word embeddings by training on a monolingual corpus and substituting translations at runtime.

# Notebook Setup

In [7]:
# general imports
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# filepaths
BASE = '~/Documents/MIDS/w266/Data' #'/home/mmillervedam/Data'
PROJ = '~/Documents/MIDS/w266/FinalProject'#'/home/mmillervedam/ProjectRepo'
FPATH_EN = BASE + '/test/wiki_en_10K.txt' # first 10000 lines from wiki dump
FPATH_ES = BASE + '/test/wiki_es_10K.txt' # first 10000 lines from wiki dump
#FULL_EN = BASE + '/en/full.txt'
#FULL_ES = BASE + '/es/full.txt'
EN_ES_DICT = PROJ +'/XlingualEmb/data/dicts/en.es.panlex.all.processed'
EN_IT_DICT  = PROJ +'/XlingualEmb/data/dicts/en.it.panlex.all.processed'
EN_IT_RAW = PROJ + '/XlingualEmb/data/mono/en_it.shuf.10k'
EN_IT_RAW = "/Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k"

# Load & Preprocess Data
__`ORIGINAL AUTHORS SAY:`__ "Normally, the monolingual word embeddings are trained on billions of words. However, getting that much of monolingual data for a low-resource language is also challenging. That is why we only select the top 5 million sentences (around 100 million words) for each language." - _Section 5.1, Duong et. al._ 

In [4]:
from parsing import Corpus, Vocabulary, batch_generator

### Corpus

In [5]:
# load corpus
en_it_data = Corpus(EN_IT_RAW)

In [6]:
# Corpus Stats
!wc {EN_IT_RAW}

   20000  430928 3746786 /Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k


__`i.e.:`__ 20K sentences (10K in each language) with ~430K tokens
> So this must not be their full data For now, I'm just going to look at the top 20K words and see what happens. In reality we should probably modify the Vocab class so that it explicily collects the top words for each language separately and then concatenates the index.

### Dictionary

In [8]:
# loading english-italian dictionary
pld = pd.read_csv(EN_IT_DICT, sep='\t', names = ['en', 'it'], dtype=str)
en_set = set(pld.en.unique())
it_set = set(pld.it.unique())

In [9]:
# dictionary vocab lengths:
print('EN:', len(en_set))
print('IT:', len(it_set))

EN: 266450
IT: 258641


### Vocabulary

In [10]:
# train multilingual Vocabulary# create vocab
en_it_vocab = Vocabulary(en_it_data.gen_tokens(), size = 100000)

### CBOW Data Generator
__`CHECK PAPER for HYPERPARAMS!`__: I can't seem to find where they talk abou the context window size, embedding size and batch size they use -- it may actually be in the Vulic and Moens paper instead of the Duong one.

In [283]:
BATCH_SIZE = 48
WINDOW_SIZE = 1
MAX_EPOCHS = 1 # fail safe

In [284]:
batches = batch_generator(en_it_data, 
                          en_it_vocab, 
                          BATCH_SIZE, 
                          WINDOW_SIZE, 
                          MAX_EPOCHS)

In [285]:
# sanity check
for context, label in batches:
    print("CONTEXT IDS:", context[:5])
    print("LABEL IDS:", label[:5])
    break

CONTEXT IDS: [[0, 1], [0, 1], [0, 34], [20, 17318], [34, 1638]]
LABEL IDS: [25668, 37957, 20, 34, 17318]


In [286]:
for context, label in batches:
    print(np.array(context))
    break

[[    0    41]
 [ 7034  7034]
 [   41    67]
 [ 7034    44]
 [   67  8276]
 [   44  9464]
 [ 8276    29]
 [ 9464    35]
 [   29  9012]
 [   35    10]
 [ 9012   149]
 [   10   310]
 [  149 13487]
 [  310   589]
 [13487    16]
 [  589 10103]
 [   16 40918]
 [10103  7024]
 [40918    10]
 [ 7024   130]
 [   10 19517]
 [  130   104]
 [19517   370]
 [  104   321]
 [  370    11]
 [  321     1]
 [    0     3]
 [   21  7801]
 [    3     6]
 [ 7801  1599]
 [    6    25]
 [ 1599 13155]
 [   25   105]
 [13155    23]
 [  105     4]
 [   23   105]
 [    4    28]
 [  105    48]
 [   28  2850]
 [   48   369]
 [ 2850   252]
 [  369     3]
 [  252  8829]
 [    3  9176]
 [ 8829     6]
 [ 9176     3]
 [    6  6342]
 [    3  2165]]


In [280]:
print(np.array.__doc__)

array(object, dtype=None, copy=True, order='K', subok=False, ndmin=0)

    Create an array.

    Parameters
    ----------
    object : array_like
        An array, any object exposing the array interface, an object whose
        __array__ method returns an array, or any (nested) sequence.
    dtype : data-type, optional
        The desired data-type for the array.  If not given, then the type will
        be determined as the minimum type required to hold the objects in the
        sequence.  This argument can only be used to 'upcast' the array.  For
        downcasting, use the .astype(t) method.
    copy : bool, optional
        If true (default), then the object is copied.  Otherwise, a copy will
        only be made if __array__ returns a copy, if obj is a nested sequence,
        or if a copy is needed to satisfy any of the other requirements
        (`dtype`, `order`, etc.).
    order : {'K', 'A', 'C', 'F'}, optional
        Specify the memory layout of the array. If object is n

# Initialize Model
__`CODE NOTES:`__ To get this running I had to hard code the context length (set to 8) inside `BuildCoreGraph()` where we generate `self.input_` in line 102. That should really be inferred from the `self.context_` itself but it doesn't seem to like the placeholder dimension (we don't have a span length until runtime). Does tensorflow not have a vectorized average? Something to fix (later). I also had to hard code the number of samples for softmax (I had originally put this as a `tf.placeholder_with_default` thinking we could pass it in to the training function (since its a training parameter) but TF kicked out an error message asking for an integer so for now I'll just give it what it wants. I need to think more about why TF doesn't want this changing from batch to batch. (or if there is another reason it wants an int).

In [337]:
from models import BiW2V

In [338]:
EMBEDDING_SIZE = 128

In [339]:
model = BiW2V(index = en_it_vocab.index, H = EMBEDDING_SIZE)

In [340]:
model.BuildCoreGraph()

In [341]:
model.BuildTrainingGraph()

In [342]:
model.BuildValidationGraph()

# Training

In [343]:
model.train(1, batches)

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(48579, 128) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(48579, 128) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(48579,) dtype=float32_ref>
... Starting Training


ValueError: Cannot feed value of shape (48,) for Tensor u'Placeholder_1:0', which has shape '(?, 1)'